In [ ]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langsmith import trace, traceable


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]

In [ ]:
import nest_asyncio

from openai import OpenAI
from typing import Dict, List
from utils import get_vector_db_retriever


RAG_SYSTEM_PROMPT = """
	You are an assistant for question-answering tasks. 
	Use the following pieces of retrieved context to answer the latest question in the conversation. 
	If you don't know the answer, just say that you don't know. 
	Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


@traceable
def retrieve_documents(question: str):
    documents = retriever.invoke(question)
    return documents


def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)	    # note: our documents came in as a list of objects, but we just want to log a string.
    with trace(
    name="Generate Response",
    run_type="chain",
    inputs={
        "question": question, 
        "formatted_docs": formatted_docs
    },
    metadata={"foo": "bar"},
    ) as ls_trace:
        messages = [
            {
                "role": "system",
                "content": RAG_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"Context: {formatted_docs} \n\n Question: {question}"
            }
        ]
        response = call_openai(messages)
        ls_trace.end(outputs={"output": response})
    return response


@traceable
def call_openai(messages: List[Dict], model: str = MODEL_NAME, temperature: float = 0.0) -> str:
    response = openai_client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature,
    )
    return response


@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 236/236 [00:43<00:00,  5.38it/s]


In [16]:
question = "How do I trace with tracing context?"
ai_answer = langsmith_rag(question)
ai_answer

"To trace with the tracing context manager in Python, you can use it to log traces for a specific block of code by wrapping that code within the context manager. This allows you to control the inputs, outputs, and other attributes of the trace. It's particularly useful when using a decorator or wrapper is not feasible."